# Lab 4: Plotting, smoothing, transformation

## Course Policies

Here are some important course policies. These are also located at
http://www.ds100.org/sp18/.

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.


## Due Date

This assignment is due at 11:59pm Monday, February 12th. Instructions for submission are on the website.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile

plt.style.use('fivethirtyeight') # Use plt.style.available to see more styles
sns.set()
sns.set_context("talk")
%matplotlib inline

## Objectives for Lab 4:

In this lab you will get some practice plotting, applying data transformations, and working with kernel density estimators.  We will be working with data from the world bank containing various statistics for countries and territories around the world.  

## Question 0

We will again need the `fetch_and_cache` utility.
Import it from `utils.py` (attached with this assignment) or redefine it below.

In [ ]:
# Import or redefine the function below

### BEGIN SOLUTION
import requests
from pathlib import Path

def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path object representing the file.
    """
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok = True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
    else:
        import time 
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version last modified (UTC):", last_modified_time)
    return file_path
### END SOLUTION

In [ ]:
# This checks whether you imported/defined fetch_and_cache
import types
assert type(fetch_and_cache) == types.FunctionType

In [ ]:
data_url = 'http://www.ds100.org/sp18/assets/datasets/lab04_data.zip'
file_name = 'lab04_data.zip'

dest_path = fetch_and_cache(data_url=data_url, file=file_name)
print(f'Located at {dest_path}')

Here, the ZIP archive contains a data folder with a few files in it. This is similar to what you had in lab 3.

To get the CSV files we want to work with, directly extract the data folder in the zip archive in the **current working directory**, which is denoted with the `.` in the cell below.

In [ ]:
my_zip = zipfile.ZipFile(dest_path, 'r')
# Note the '.' argument
my_zip.extractall('.')

Now, let us load some world bank data into a pandas.DataFrame object named ```wb```.

In [ ]:
wb = pd.read_csv("data/world_bank_misc.csv", index_col=0)
wb.head()

This table contains some interesting columns.  Take a look:

In [ ]:
list(wb.columns)

# Part 1: Scaling

In the first part of this assignment we will be scaling the data to linearize visualizations.


## Question 1:

Extract the fields corresponding to the **adult literacy rate in Female ages 15 and older for 2005-14** and the **gross national income per capita (atlas method)** into a new dataframe.  Then drop any rows that are missing values.

In [ ]:
df = pd.DataFrame(index=wb.index)
df['lit'] = ...
df['inc'] = ...

### BEGIN SOLUTION
df['lit'] = wb['Adult literacy rate: Female: % ages 15 and older: 2005-14']
df['inc'] = wb['Gross national income per capita, Atlas method: $: 2016']
### END SOLUTION

df.dropna(inplace=True)
print("Original records:", len(wb))
print("Final records:", len(df))

In [ ]:
assert np.isclose(df['lit'].mean(),78.435, rtol=0.01)
assert np.isclose(df['inc'].mean(),7919.251, rtol=0.01)

## Question 2a:

Use the seaborn `distplot` tool to construct histograms for the adult literacy data and the income data:

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
# Make plot here

### BEGIN SOLUTION
sns.distplot(df['lit'])
### END SOLUTION

plt.xlabel("Adult literacy rate: Female: % ages 15 and older: 2005-14")



plt.subplot(1,2,2)
# Make plot here

### BEGIN SOLUTION
sns.distplot(df['inc'])
### END SOLUTION

plt.xlabel('Gross national income per capita, Atlas method: $: 2016')


## Question 2b

One of the above plots could benefit from a log transformation.  Which one?

In [ ]:
needs_log_transformation = ... # answer "lit" or "inc" here

### BEGIN SOLUTION
needs_log_transformation = "inc"
### END SOLUTION

In [ ]:
# Do not modify this cell
### BEGIN HIDDEN TESTS
assert needs_log_transformation == "inc"
### END HIDDEN TESTS

## Question 2c

Remake the appropriate plot with the data transformed using `log10`. Be sure to correct the axis label:

In [ ]:
plt.figure()
...

### BEGIN SOLUTION
sns.distplot(np.log10(df['inc']))
plt.xlabel('Log Gross national income per capita, Atlas method: $: 2016')
### END SOLUTION

# Part 2: Kernel Density Estimation

In this part of the lab you will implement a kernel density estimator.


Let's implement our own version of the KDE plot above.  Below we give you the Guassian Kernel function

$$\Large
K_\alpha(x, z) = \frac{1}{\sqrt{2 \pi \alpha^2}} \exp\left(-\frac{(x - z)^2}{2  \alpha ^2} \right)
$$

In [ ]:
def gaussian_kernel(alpha, x, z):
    return 1.0/np.sqrt(2. * np.pi * alpha**2) * np.exp(-(x - z) ** 2 / (2.0 * alpha**2))

## Question 3a
Implement the KDE function which computes:

$$\Large
f_\alpha(x) = \frac{1}{n} \sum_{i=1}^n K_\alpha(x, z_i)
$$

Where $z_i$ are the data and $\alpha$ is a parameter to control the smoothness

In [ ]:
def kde(kernel, alpha, x, data):
    """
    Compute the kernel density estimate for the single query point x.

    Args:
        kernel: a kernel function with 3 parameters: alpha, x, data
        alpha: the smoothing parameter to pass to the kernel
        x: a single query point (in one dimension)
        data: a numpy array of data points

    Returns:
        The smoothed estimate at the query point x
    """
    ...
    
    ### BEGIN SOLUTION
    return np.mean(kernel(alpha, data, x), axis=0)
    ### END SOLUTION

In [ ]:
assert np.isclose(kde(gaussian_kernel, 1.0, 2.0, np.array([3.0, 4.0, 5.0, 7.0])), 0.075099)

## Question 3b
Create two new columns `trans_lit` and `trans_inc` that transform the `lit` and `inc` columns using `log10`. This should be similar to what you did in Question 2c.


In [ ]:
df['trans_lit'] = df['lit'] # Change me
df['trans_inc'] = df['inc'] # Change me

### BEGIN SOLUTION
df['trans_lit'] = np.log10(df['lit'])
df['trans_inc'] = np.log10(df['inc'])
### END SOLUTION

In [ ]:
assert np.isclose(np.corrcoef(df['trans_lit'], df['trans_inc'])[0,1], 0.67196)

Now let's test your function to generate a plot. You may find the ```np.linspace``` function helpful when plotting the KDE curve.

In [ ]:
alpha = 1.0
xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
curve = [kde(gaussian_kernel, alpha, x, df['trans_inc']) for x in xs]
plt.hist(df['trans_inc'], normed=True, color='orange')
plt.plot(xs, curve, 'k-')

## Question 3c

Let's see what happens as we vary alpha.  Plot alpha values in (0.2, 0.4, ..., 1.6, 1.8) on a 3x3 grid. Title each subplot with "alpha = &lt;value&gt;".

Hint: Take a look at the `plt.subplot` function to help create the grid

In [ ]:
plt.figure(figsize=(15,15))
alphas = ...
for i, alpha in enumerate(alphas):
    ...
    xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
    curve = [kde(gaussian_kernel, alpha, x, df['trans_inc']) for x in xs]
    plt.hist(df['trans_inc'], normed=True, color='orange')
    plt.plot(xs, curve, 'k-')
plt.show()

### BEGIN SOLUTION
plt.figure(figsize=(15,15))
alphas = np.arange(0.2, 1.9, 0.2)
for i, alpha in enumerate(np.arange(0.2, 1.9, 0.2)):
    plt.subplot(3, 3, i+1)
    xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
    curve = [kde(gaussian_kernel, alpha, x, df['trans_inc']) for x in xs]
    plt.hist(df['trans_inc'], normed=True, color='orange')
    plt.plot(xs, curve, 'k-')
    plt.title("alpha = " + str(alpha))
plt.show()
### END SOLUTION

How does increasing alpha affect the curves? Explain any pattern/trend you see in the cell below. 

Larger alpha will have more smooth curve. 
But if we use an alpha that is too large, each point on the curve will get very small value. (Like a flat curve)
The scale of alpha can be set to match the scale of the length for each histogram bin.

## Question 3d

We can also try other kernel functions such as the [boxcar kernel](https://en.wikipedia.org/wiki/Boxcar_function).


In [ ]:
def boxcar_kernel(alpha, x, z):
    return (((x-z)>=-alpha/2)&((x-z)<=alpha/2))/alpha

Run the cell below to enable interactive plots. It should give you a green 'OK' when it's finished.

In [ ]:
from ipywidgets import interact
!jupyter nbextension enable --py widgetsnbextension

Now, we can plot the kernel function to see what it looks like.

In [ ]:
x = np.linspace(-10,10,1000)
def f(alpha):
    plt.plot(x, boxcar_kernel(alpha,x,0), label='Boxcar')
    plt.plot(x, gaussian_kernel(alpha,x,0), label='Gaussian')
    plt.legend(title='Kernel Function')
    plt.show()
interact(f, alpha=(1,10,0.1))

Using the interactive plot below compare the the two kernel techniques:  (Generating the KDE plot is slow, so you may expect some latency after you move the slider)

In [ ]:
xs = np.linspace(df['trans_inc'].min(), df['trans_inc'].max(), 1000)
def f(alpha_g, alpha_b):
    plt.hist(df['trans_inc'], normed=True, color='orange')
    g_curve = [kde(gaussian_kernel, alpha_g, x, df['trans_inc']) for x in xs]
    plt.plot(xs, g_curve, 'k-', label='Gaussian')
    b_curve = [kde(boxcar_kernel, alpha_b, x, df['trans_inc']) for x in xs]
    plt.plot(xs, b_curve, 'r-', label='Boxcar')
    plt.legend(title='Kernel Function')
    plt.show()
interact(f, alpha_g=(0.01,.5,0.01), alpha_b=(0.01,3,0.1))

How is the boxcar kde plot comparing to previous plot using the gaussian kernel?

The gaussian kde is more smooth. 

**Congrats! You are finished with this assignment. Don't forget to validate & submit before 11:59PM!**